# Expanding R-Group Substructures from P2M

The Proteome2Metabolome (P2M) tool is useful in translating proteome identifiers into computer-friendly lists of associated metabolite compounds for the purpose of upstream compound identification. However, some queried metabolite structures are represented as R-group substructures of larger protein scaffolds or biomolecules.

To expand the R-group containing substructures into potential full structures, users can process their P2M output as follows.

In [7]:
# import libraries
import p2m
import pandas as pd

In [8]:
# load R-group structures (containing '*' wildcards)
data = pd.read_csv("../data/example_run/p2m/partial/smiles_partial.tsv", sep="\t")
print(len(data))
data.head()

360


,uniprot,identifier,chebiId,name,smiles,rheaIds
0,http://purl.uniprot.org/uniprot/P0AGG4,P0AGG4,CHEBI:29950,[protein]-dithiol,C(=O)(*)[C@@H](N*)CS,"18749,18749,18753,18753"
1,http://purl.uniprot.org/uniprot/P0AGG4,P0AGG4,CHEBI:50058,[protein]-disulfide,C([C@@H](N*)CSSC[C@@H](C(=O)*)N*)(=O)*,"18749,18753"
2,http://purl.uniprot.org/uniprot/P68398,P68398,CHEBI:74411,adenosine(34) in tRNA,NC1=NC=NC2=C1N=CN2[C@@H]3O[C@H](COP(=O)(*)[O-]...,43168
3,http://purl.uniprot.org/uniprot/P68398,P68398,CHEBI:82852,inosine(34) in tRNA,O[C@@H]1[C@H](O-*)[C@@H](COP([O-])(-*)=O)O[C@H...,43168
4,http://purl.uniprot.org/uniprot/P0A9K9,P0A9K9,CHEBI:83834,[protein]-peptidylproline (omega=180),*-C(=O)[C@@H]1CCCN1C(-*)=O,16237


In [12]:
# take first partial smiles
i = 0
smiles = data["smiles"].values[i]

result = p2m.query.substruct2struct(i)
print(len(result))
result.head()

2609


,chebiId,name,smiles
0,CHEBI:159602,Asn-Asp-Cys,SC[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)CC(=O)N)CC...
1,CHEBI:39390,felinine,CC(C)(CCO)SC[C@H](N)C(O)=O
2,CHEBI:53600,benzylpenicillanyl group,[H][C@]12SC(C)(C)[C@@H](N1C(=O)[C@H]2NC(=O)CC3...
3,CHEBI:135919,nesiritide,[H]N[C@@H](CO)C(N1CCC[C@H]1C(N[C@@H](CCCCN)C(N...
4,CHEBI:159670,Asn-Cys-Tyr,SC[C@H](NC(=O)[C@@H](N)CC(=O)N)C(=O)N[C@@H](CC...
